In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [6]:
df_atraso = pd.read_csv(r'BD Tratado\atraso.csv',sep = ';')
df_atraso = df_atraso[['MATRÍCULA', 'DATA', 'ATRASO']]
df_atraso = df_atraso.sort_values(['MATRÍCULA', 'DATA'])
df_atraso['DATA'] = pd.to_datetime(df_atraso['DATA'])
df_atraso = df_atraso[df_atraso['ATRASO'] != 0]
df_atraso.head()

,MATRÍCULA,DATA,ATRASO
5422,3108,2020-01-03,1.0
13922,3108,2020-01-10,72.0
67987,3108,2020-01-20,46.0
86047,3108,2020-01-21,47.0
27640,3108,2020-01-22,32.0


In [2]:
df_carga = pd.read_csv(r'BD Tratado\carga.csv',sep=';')
df_carga = df_carga.sort_values(['MATRÍCULA', 'DATA'])
df_carga['DATA FIM'] = np.where(df_carga['MATRÍCULA'].shift(-1) == df_carga['MATRÍCULA'], df_carga['DATA'].shift(-1), '2021-04-30')
df_carga['DATA FIM'] = pd.to_datetime(df_carga['DATA FIM'])
df_carga['DATA'] = pd.to_datetime(df_carga['DATA'])
df_carga.head()

,MATRÍCULA,DATA,CRG. HOR. DIA,DATA FIM
16225,3108,2020-02-15,NaN,2020-06-10
12967,3108,2020-06-10,08:30,2021-01-29
3447,3108,2021-01-29,08:48,2021-04-30
16923,3143,2020-01-24,08:00,2020-05-21
13609,3143,2020-05-21,NaN,2021-01-29


In [3]:
df_carga.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17593 entries, 16225 to 2685
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   MATRÍCULA      17593 non-null  int64         
 1   DATA           17593 non-null  datetime64[ns]
 2   CRG. HOR. DIA  13799 non-null  object        
 3   DATA FIM       17593 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 687.2+ KB


In [4]:
df_jornada = pd.read_csv(r'BD Tratado\horarios.csv', sep = ';')
df_jornada.drop(['Unnamed: 0', 'HORÁRIO'], axis=1, inplace=True)
df_jornada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   MATRÍCULA        164 non-null    int64
 1   Jornada - 5x2    164 non-null    int64
 2   Jornada - 6x1    164 non-null    int64
 3   Jornada - 12x36  164 non-null    int64
 4   Jornada - movel  164 non-null    int64
 5   Jornada - fixo   164 non-null    int64
dtypes: int64(6)
memory usage: 7.8 KB


In [5]:
df_jornada.head()

,MATRÍCULA,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo
0,205827,0,1,0,1,0
1,90293,1,0,0,0,1
2,200400,1,0,0,0,1
3,200570,1,0,0,0,1
4,200981,0,1,0,1,0


In [7]:
df_noturno = pd.read_csv(r'BD Tratado\noturno.csv',sep=';')
df_noturno = df_noturno.sort_values(['MATRÍCULA', 'DATA'])
df_noturno['DATA FIM'] = np.where(df_noturno['MATRÍCULA'].shift(-1) == df_noturno['MATRÍCULA'], df_noturno['DATA'].shift(-1), '2021-04-30')
df_noturno['DATA FIM'] = pd.to_datetime(df_noturno['DATA FIM'])
df_noturno['DATA'] = pd.to_datetime(df_noturno['DATA'])
df_noturno.head(10)

,MATRÍCULA,DATA,NOTURNO,DATA FIM
11163,3108,2020-02-13,0,2020-02-14
11127,3108,2020-02-14,0,2020-06-10
9616,3108,2020-06-10,0,2020-06-30
9265,3108,2020-06-30,0,2020-07-10
9097,3108,2020-07-10,0,2020-07-30
8760,3108,2020-07-30,0,2020-08-26
8263,3108,2020-08-26,0,2021-01-29
3167,3108,2021-01-29,0,2021-04-30
11398,3143,2020-01-24,0,2021-01-29
2775,3143,2021-01-29,0,2021-04-30


In [8]:
df = df_atraso.merge(df_jornada, how = 'left', on = 'MATRÍCULA')
df.head()

,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo
0,3108,2020-01-03,1.0,NaN,NaN,NaN,NaN,NaN
1,3108,2020-01-10,72.0,NaN,NaN,NaN,NaN,NaN
2,3108,2020-01-20,46.0,NaN,NaN,NaN,NaN,NaN
3,3108,2020-01-21,47.0,NaN,NaN,NaN,NaN,NaN
4,3108,2020-01-22,32.0,NaN,NaN,NaN,NaN,NaN


In [9]:
#Make the db in memory
conn = sqlite3.connect(':memory:')
#write the tables
df.to_sql('df', conn, index=False)
df_carga.to_sql('df_carga', conn, index=False)

qry = '''
    select  
        t1.*,
        t2.[CRG. HOR. DIA]
        
    from
        df as t1
        left join df_carga as t2
            on t1.DATA between t2.DATA and t2.[DATA FIM] 
            and t1.MATRÍCULA = t2.MATRÍCULA
        '''
df = pd.read_sql_query(qry, conn)
df.head()

C:\Users\gabri\anaconda3\envs\faculdade\lib\site-packages\pandas\core\generic.py:2789: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo,CRG. HOR. DIA
0,3108,2020-01-03 00:00:00,1.0,NaN,NaN,NaN,NaN,NaN,None
1,3108,2020-01-10 00:00:00,72.0,NaN,NaN,NaN,NaN,NaN,None
2,3108,2020-01-20 00:00:00,46.0,NaN,NaN,NaN,NaN,NaN,None
3,3108,2020-01-21 00:00:00,47.0,NaN,NaN,NaN,NaN,NaN,None
4,3108,2020-01-22 00:00:00,32.0,NaN,NaN,NaN,NaN,NaN,None


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98310 entries, 0 to 98309
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   MATRÍCULA        98310 non-null  int64  
 1   DATA             98310 non-null  object 
 2   ATRASO           98310 non-null  float64
 3   Jornada - 5x2    2726 non-null   float64
 4   Jornada - 6x1    2726 non-null   float64
 5   Jornada - 12x36  2726 non-null   float64
 6   Jornada - movel  2726 non-null   float64
 7   Jornada - fixo   2726 non-null   float64
 8   CRG. HOR. DIA    30927 non-null  object 
dtypes: float64(6), int64(1), object(2)
memory usage: 6.8+ MB


In [11]:
#Make the db in memory
conn = sqlite3.connect(':memory:')
#write the tables
df.to_sql('df', conn, index=False)
df_noturno.to_sql('df_noturno', conn, index=False)

qry = '''
    select  
        t1.*,
        t2.[NOTURNO]
        
    from
        df as t1
        left join df_noturno as t2
            on t1.DATA between t2.DATA and t2.[DATA FIM] 
            and t1.MATRÍCULA = t2.MATRÍCULA
        '''
df = pd.read_sql_query(qry, conn)
df.head()

C:\Users\gabri\anaconda3\envs\faculdade\lib\site-packages\pandas\core\generic.py:2789: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo,CRG. HOR. DIA,NOTURNO
0,3108,2020-01-03 00:00:00,1.0,NaN,NaN,NaN,NaN,NaN,None,NaN
1,3108,2020-01-10 00:00:00,72.0,NaN,NaN,NaN,NaN,NaN,None,NaN
2,3108,2020-01-20 00:00:00,46.0,NaN,NaN,NaN,NaN,NaN,None,NaN
3,3108,2020-01-21 00:00:00,47.0,NaN,NaN,NaN,NaN,NaN,None,NaN
4,3108,2020-01-22 00:00:00,32.0,NaN,NaN,NaN,NaN,NaN,None,NaN


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100557 entries, 0 to 100556
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   MATRÍCULA        100557 non-null  int64  
 1   DATA             100557 non-null  object 
 2   ATRASO           100557 non-null  float64
 3   Jornada - 5x2    2791 non-null    float64
 4   Jornada - 6x1    2791 non-null    float64
 5   Jornada - 12x36  2791 non-null    float64
 6   Jornada - movel  2791 non-null    float64
 7   Jornada - fixo   2791 non-null    float64
 8   CRG. HOR. DIA    32638 non-null   object 
 9   NOTURNO          51540 non-null   float64
dtypes: float64(7), int64(1), object(2)
memory usage: 7.7+ MB


In [13]:
#analisando um caso
df[df['MATRÍCULA'] == 3108]

,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo,CRG. HOR. DIA,NOTURNO
0,3108,2020-01-03 00:00:00,1.0,NaN,NaN,NaN,NaN,NaN,None,NaN
1,3108,2020-01-10 00:00:00,72.0,NaN,NaN,NaN,NaN,NaN,None,NaN
2,3108,2020-01-20 00:00:00,46.0,NaN,NaN,NaN,NaN,NaN,None,NaN
3,3108,2020-01-21 00:00:00,47.0,NaN,NaN,NaN,NaN,NaN,None,NaN
4,3108,2020-01-22 00:00:00,32.0,NaN,NaN,NaN,NaN,NaN,None,NaN
5,3108,2020-01-23 00:00:00,36.0,NaN,NaN,NaN,NaN,NaN,None,NaN
6,3108,2020-01-24 00:00:00,46.0,NaN,NaN,NaN,NaN,NaN,None,NaN
7,3108,2020-02-03 00:00:00,314.0,NaN,NaN,NaN,NaN,NaN,None,NaN
8,3108,2020-02-10 00:00:00,150.0,NaN,NaN,NaN,NaN,NaN,None,NaN
9,3108,2020-02-14 00:00:00,170.0,NaN,NaN,NaN,NaN,NaN,None,0.0


In [14]:
df.to_csv(r'BD Tratado\base.csv',sep=';',index=False)